In [1]:
import pandas as pd

books=pd.read_csv("books_categorised.csv")

In [2]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True, top_k=None)
classifier("I love this!")

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.005764591973274946},
  {'label': 'anger', 'score': 0.004419785924255848},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.0004138524236623198}]]

In [3]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548399329185486},
  {'label': 'neutral', 'score': 0.1698525846004486},
  {'label': 'sadness', 'score': 0.11640949547290802},
  {'label': 'surprise', 'score': 0.020700670778751373},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.015161491930484772},
  {'label': 'anger', 'score': 0.003935153596103191}]]

In [4]:
predictions=classifier(books["description"][0].split("."))
#sentence wise analysis for finer classification of overall tone
predictions

[[{'label': 'surprise', 'score': 0.7296022176742554},
  {'label': 'neutral', 'score': 0.14038586616516113},
  {'label': 'fear', 'score': 0.06816234439611435},
  {'label': 'joy', 'score': 0.047942597419023514},
  {'label': 'anger', 'score': 0.009156372398138046},
  {'label': 'disgust', 'score': 0.002628477057442069},
  {'label': 'sadness', 'score': 0.002122163074091077}],
 [{'label': 'neutral', 'score': 0.44936978816986084},
  {'label': 'disgust', 'score': 0.273592084646225},
  {'label': 'joy', 'score': 0.10908310860395432},
  {'label': 'sadness', 'score': 0.09362740814685822},
  {'label': 'anger', 'score': 0.04047837108373642},
  {'label': 'surprise', 'score': 0.026970159262418747},
  {'label': 'fear', 'score': 0.006879053078591824}],
 [{'label': 'neutral', 'score': 0.6462159156799316},
  {'label': 'sadness', 'score': 0.24273331463336945},
  {'label': 'disgust', 'score': 0.043422672897577286},
  {'label': 'surprise', 'score': 0.0283005703240633},
  {'label': 'joy', 'score': 0.014211491

In [5]:
sorted(predictions[0], key=lambda x:x["label"])

[{'label': 'anger', 'score': 0.009156372398138046},
 {'label': 'disgust', 'score': 0.002628477057442069},
 {'label': 'fear', 'score': 0.06816234439611435},
 {'label': 'joy', 'score': 0.047942597419023514},
 {'label': 'neutral', 'score': 0.14038586616516113},
 {'label': 'sadness', 'score': 0.002122163074091077},
 {'label': 'surprise', 'score': 0.7296022176742554}]

In [13]:
import numpy as np

emotion_labels =["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn=[]
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores={label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions=sorted(prediction, key=lambda x:x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [14]:
from tqdm import tqdm

for i in tqdm(range(10)):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions=classifier(sentences)
    max_scores=calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 10/10 [00:01<00:00,  6.00it/s]


In [15]:
emotion_scores

{'anger': [np.float64(0.0641336739063263),
  np.float64(0.6126192212104797),
  np.float64(0.0641336739063263),
  np.float64(0.3514849543571472),
  np.float64(0.08141250908374786),
  np.float64(0.23222453892230988),
  np.float64(0.5381843447685242),
  np.float64(0.0641336739063263),
  np.float64(0.3006702661514282),
  np.float64(0.0641336739063263)],
 'disgust': [np.float64(0.273592084646225),
  np.float64(0.348284512758255),
  np.float64(0.10400678217411041),
  np.float64(0.15072233974933624),
  np.float64(0.1844957321882248),
  np.float64(0.7271750569343567),
  np.float64(0.15585479140281677),
  np.float64(0.10400678217411041),
  np.float64(0.2794811725616455),
  np.float64(0.1779264211654663)],
 'fear': [np.float64(0.9281684160232544),
  np.float64(0.9425276517868042),
  np.float64(0.9723207950592041),
  np.float64(0.3607054650783539),
  np.float64(0.09504339098930359),
  np.float64(0.051362860947847366),
  np.float64(0.7474274039268494),
  np.float64(0.40449756383895874),
  np.float

In [16]:
from tqdm import tqdm

emotion_labels =["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn=[]
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions=classifier(sentences)
    max_scores=calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

  0%|          | 0/5197 [00:00<?, ?it/s]

100%|██████████| 5197/5197 [11:21<00:00,  7.63it/s]


In [17]:
emotions_df=pd.DataFrame(emotion_scores)
emotions_df["isbn13"]=isbn

In [18]:
emotions_df.head()

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928168,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612619,0.348285,0.942528,0.704422,0.887940,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767239,0.549477,0.111690,0.078765,9780006178736
3,0.351485,0.150722,0.360705,0.251881,0.732685,0.111690,0.078765,9780006280897
4,0.081413,0.184496,0.095043,0.040564,0.884390,0.475880,0.078765,9780006280934


In [19]:
books=pd.merge(books, emotions_df, on="isbn13", how="left")

In [20]:
books.to_csv("books_with_emotions.csv", index=False)